<a href="https://colab.research.google.com/github/Prakrut/Fraud_Detection/blob/master/Code/CreditCardFraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In [ ]:
print("Hello from Prakrut!")

Hello from Prakrut!
